In [1]:
import brightway2 as bw
from premise import *
import pandas as pd
from itertools import product
import matplotlib.pyplot as plt
bw.projects.set_current("carculator")

In [10]:
if "ben_db_0" in bw.databases:
    del bw.databases["ben_db_0"]

Vacuuming database 


In [5]:
clear_cache()

Cache folder cleared!


In [2]:
combinations = [
    {"range time":False, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 5, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":True, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":True, "lead time":False, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 1, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 2, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 3, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":True, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":True, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":True, "lead time":False, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 1, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 2, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 3, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":True, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 4, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":True, "lead time":False, "capital replacement rate":False, "measurement": 4, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":True, "measurement": 4, "weighted slope start": 0.75, "weighted slope end": 1.00}
]

In [8]:
scenarios = [
    {"model": "remind", "pathway":"SSP2-PkBudg500", "year": 2030},
    {"model": "remind", "pathway":"SSP2-Base", "year": 2050},
    {"model": "image", "pathway":"SSP2-Base", "year": 2040},
    {"model": "image", "pathway":"SSP2-Base", "year": 2020},
]

l_res = []

for a, args in enumerate(combinations):
    ndb = NewDatabase(
            scenarios = scenarios,        
            source_db="ecoinvent 3.8 cutoff",
            source_version="3.8",
            key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
            system_model="consequential",
            system_args = args
        )
    #ndb.update_all()
    #ndb.write_superstructure_db_to_brightway(name=f"ben_db_{a}")
    l_res.append(
        (ndb.iam_data.electricity_markets.sel(region="CHA", year=2050).sum(dim="scenario")
        ).to_dataframe("val")["val"].values.tolist()
    )


premise v.(2, 0, 0)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [10]:
pd.DataFrame(l_res, columns=ndb.iam_data.electricity_markets.variables.values.tolist()).to_excel("results_CHA_2050_Base.xlsx")

In [13]:
for cs in list(bw.calculation_setups.keys()):
    del bw.calculation_setups[cs]

In [14]:
for i in range(0, 19):
    acts = [{(a["database"], a["code"]):1} for a in bw.Database(f"ben_db_{i}") if a["name"] == "market group for electricity, low voltage" and a["location"] in ["EUR", "CHA", "WEU"]]
    meth = ('IPCC 2013', 'climate change', 'GWP 100a')
    
    bw.calculation_setups[f"ben_{i}"] = {
        'inv': acts,
        'ia': [meth]}

In [15]:
bw.calculation_setups["ben_0"]

{'inv': [{('ben_db_0', '9498048180618909048101470389'): 1},
  {('ben_db_0', '4143946432033982059594954568'): 1},
  {('ben_db_0', '56883951199402670991359028683'): 1}],
 'ia': [('IPCC 2013', 'climate change', 'GWP 100a')]}

In [ ]:
ndb.get

In [1]:
import brightway2 as bw
from premise import *
import pandas as pd
from itertools import product
import matplotlib.pyplot as plt
bw.projects.set_current("premise")

args = {'range time': False, 'duration': False, 'foresight': True, 'lead time': True, 'capital replacement rate': False, 'measurement': 2}

ndb = NewDatabase(
    scenarios = [
        {"model":"image", "pathway":"SSP2-RCP19", "year":2030},

    ],        
    source_db="ecoinvent 3.8 cutoff",
    source_version="3.8",
    key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
    system_model="consequential",
    system_args = args
)

+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+

////////////////////// EXTRACTING SOURCE DATABASE //////////////////
Done!

//////////////////// IMPOR

In [ ]:
import xarray as xr

In [2]:
import numpy as np

In [ ]:
a = np.array([1, 2, 3, 4, ])

In [4]:
from country_converter import CountryConverter
coco = CountryConverter()

In [8]:
coco.ISO2.ISO2.values.tolist()

['AF',
 'AX',
 'AL',
 'DZ',
 'AS',
 'AD',
 'AO',
 'AI',
 'AQ',
 'AG',
 'AR',
 'AM',
 'AW',
 'AU',
 'AT',
 'AZ',
 'BS',
 'BH',
 'BD',
 'BB',
 'BY',
 'BE',
 'BZ',
 'BJ',
 'BM',
 'BT',
 'BO',
 'BQ',
 'BA',
 'BW',
 'BV',
 'BR',
 'IO',
 'VG',
 'BN',
 'BG',
 'BF',
 'BI',
 'CV',
 'KH',
 'CM',
 'CA',
 'KY',
 'CF',
 'TD',
 'CL',
 'CN',
 'CX',
 'CC',
 'CO',
 'KM',
 'CG',
 'CK',
 'CR',
 'CI',
 'HR',
 'CU',
 'CW',
 'CY',
 'CZ',
 'DK',
 'DJ',
 'DM',
 'DO',
 'CD',
 'EC',
 'EG',
 'SV',
 'GQ',
 'ER',
 'EE',
 'SZ',
 'ET',
 'FO',
 'FK',
 'FJ',
 'FI',
 'FR',
 'GF',
 'PF',
 'TF',
 'GA',
 'GM',
 'GE',
 'DE',
 'GH',
 'GI',
 'GR',
 'GL',
 'GD',
 'GP',
 'GU',
 'GT',
 'GG',
 'GN',
 'GW',
 'GY',
 'HT',
 'HM',
 'HN',
 'HK',
 'HU',
 'IS',
 'IN',
 'ID',
 'IR',
 'IQ',
 'IE',
 'IM',
 'IL',
 'IT',
 'JM',
 'JP',
 'JE',
 'JO',
 'KZ',
 'KE',
 'KI',
 'XK',
 'KW',
 'KG',
 'LA',
 'LV',
 'LB',
 'LS',
 'LR',
 'LY',
 'LI',
 'LT',
 'LU',
 'MO',
 'MK',
 'MG',
 'MW',
 'MY',
 'MV',
 'ML',
 'MT',
 'MH',
 'MQ',
 'MR',
 'MU',
 'YT',